In [1]:
from scipy.signal import butter, lfilter
import scipy
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
import time
import torch.nn.functional as F

# Load dataset

In [2]:
data_path = "../Data/Physionet_processed/"

In [3]:
dataset_1=np.load(data_path + '1.npy')
print('The shape of Dataset_1:', dataset_1.shape)
dataset_1

The shape of Dataset_1: (259520, 65)


array([[-16, -29,   2, ..., -11,  15,   0],
       [-56, -54, -27, ...,   1,  21,   0],
       [-55, -55, -29, ...,  18,  35,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9]], dtype=int64)

# Filtering

In [4]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = scipy.signal.lfilter(b, a, data)
    return y

In [5]:
n_fea = 64  # 64 channels
label = dataset_1[:, n_fea: n_fea+1]  # seperate label from feature
feature = dataset_1[:, 0:n_fea]
feature_f=[]  # feature after filtering

# EEG Delta pattern decomposition
for i in range(feature.shape[1]):
    x = feature[:, i]
    fs = 160.0
    lowcut = 0.5
    highcut = 4.0
    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=3)
    feature_f.append(y)

feature_f=np.array(feature_f).T
print('The shape of filtered feature:',feature_f.shape)

data_f=np.hstack((feature_f,label))  # stack label to filtered feature
print("The shape of dataset_1 after filtering:",data_f.shape)

The shape of filtered feature: (259520, 64)
The shape of dataset_1 after filtering: (259520, 65)


In [6]:
def extract(input, n_classes, n_fea, time_window, moving):
    xx = input[:, :n_fea]
    yy = input[:, n_fea:n_fea + 1]
    new_x = []
    new_y = []
    number = int((xx.shape[0] / moving) - 1)
    for i in range(number):
        ave_y = np.average(yy[int(i * moving):int(i * moving + time_window)])
        if ave_y in range(n_classes + 1):
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(ave_y)
        else:
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(0)

    new_x = np.array(new_x)
    new_x = new_x.reshape([-1, n_fea * time_window])
    new_y = np.array(new_y)
    new_y.shape = [new_y.shape[0], 1]
    data = np.hstack((new_x, new_y))
    data = np.vstack((data, data[-1]))  # add the last sample again, to make the sample number round
    return data

In [9]:
n_class = 10  # 0~9 classes ('10:rest' is not considered)
segment_length = 3  # selected time window; 16=160*0.1

# segment data, check more details about the 'extract' function in BCI_functions.ipynb
data_seg = extract(dataset_1, n_classes=n_class, n_fea=n_fea, time_window=segment_length, moving=(segment_length/2))  # 50% overlapping

print('After segmentation, the shape of the data:', data_seg.shape)

After segmentation, the shape of the data: (173013, 193)


In [10]:
# remove instance with label==10 (rest)
# removed_label = [2,3,4,5,6,7,8,9,10]  #2,3,4,5,
# for ll in removed_label:
#     id = dataset_1[:, -1]!=ll
#     dataset_1 = dataset_1[id]

# data segmentation
# n_class = int(11-len(removed_label))  # 0~9 classes ('10:rest' is not considered)
no_feature = 64  # the number of the features
segment_length = 3  # selected time window; 16=160*0.1
LR = 0.005  # learning rate
EPOCH = 401

data_seg = extract(dataset_1, n_classes=n_class, n_fea=no_feature, time_window=segment_length, moving=(segment_length/2))  # 50% overlapping
print('After segmentation, the shape of the data:', data_seg.shape)


After segmentation, the shape of the data: (173013, 193)


In [11]:
# split training and test data
no_longfeature = no_feature*segment_length
data_seg_feature = data_seg[:, :no_longfeature]
data_seg_label = data_seg[:, no_longfeature:no_longfeature+1]
train_feature, test_feature, train_label, test_label = train_test_split(data_seg_feature, data_seg_label, shuffle=True)

In [12]:
train_feature

array([[ -30.,  -18.,    3., ..., -115.,  -88.,  -96.],
       [ -26.,  -17.,   -6., ...,  -47.,  -55.,  -10.],
       [  40.,   48.,   29., ...,   58.,   92.,   59.],
       ...,
       [  58.,   58.,   66., ...,   24.,  -20.,   14.],
       [  43.,   77.,   88., ...,  -15.,    6.,  -92.],
       [ -84.,  -68.,  -56., ...,   33.,   51.,   19.]])

In [13]:
test_feature

array([[ 13.,  14., -11., ...,  -8., -54., -96.],
       [-50., -48., -37., ...,  12.,  54., 124.],
       [-23., -20., -40., ..., -22.,  -1., -12.],
       ...,
       [-36., -26., -18., ...,  32., -32., -13.],
       [  9., -32., -28., ..., -43., -32.,   2.],
       [ 72.,  58.,  48., ...,  44.,  30.,  40.]])

# Preprocessing

In [14]:
# normalization
# before normalize reshape data back to raw data shape
train_feature_2d = train_feature.reshape([-1, no_feature])
test_feature_2d = test_feature.reshape([-1, no_feature])

# min-max normalization
scaler3 = MinMaxScaler().fit(train_feature)
train_fea_norm1 = scaler3.transform(train_feature)
test_fea_norm1 = scaler3.transform(test_feature)
print('After normalization, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter normalization, the shape of test feature:', test_fea_norm1.shape)

# after normalization, reshape data to 3d in order to feed in to LSTM
train_fea_norm1 = train_fea_norm1.reshape([-1, segment_length, no_feature])
test_fea_norm1 = test_fea_norm1.reshape([-1, segment_length, no_feature])
print('After reshape, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter reshape, the shape of test feature:', test_fea_norm1.shape)

After normalization, the shape of training feature: (129759, 192) 
After normalization, the shape of test feature: (43254, 192)
After reshape, the shape of training feature: (129759, 3, 64) 
After reshape, the shape of test feature: (43254, 3, 64)


In [15]:
BATCH_size = train_fea_norm1.shape[0] # use test_data as batch size

# feed data into dataloader
train_fea_norm1 = torch.tensor(train_fea_norm1).type('torch.FloatTensor')
train_label = torch.tensor(train_label.flatten())
train_data = Data.TensorDataset(train_fea_norm1, train_label)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_size, shuffle=False)

test_fea_norm1 = torch.tensor(test_fea_norm1).type('torch.FloatTensor')
test_label = torch.tensor(test_label.flatten())

## Feature Extraction

In [16]:
def one_hot(y_):
    y_ = y_.reshape(len(y_))
    y_ = [int(xx) for xx in y_]
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]
    import numpy as np

In [17]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(no_feature*segment_length, 64*4),
        )
        self.decoder = nn.Sequential(
            nn.Linear(64*4, no_feature*segment_length),
            nn.Sigmoid(),
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

autoencoder = AutoEncoder()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR)
loss_func = nn.MSELoss()

best_acc = []

## Classifiers

In [19]:
# n_class = int(11-len(removed_label))  # 0~9 classes ('10:rest' is not considered)
no_feature = 64  # the number of the features
segment_length = 3  # selected time window; 16=160*0.1
LR = 0.005  # learning rate
EPOCH = 101
n_hidden = 128  # number of neurons in hidden layer
l2 = 0.001  # the coefficient of l2-norm regularization

## LSTM

In [20]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()

        self.lstm_layer = nn.LSTM(
            input_size=no_feature,
            hidden_size=n_hidden,         # LSTM hidden unit
            num_layers=2,           # number of LSTM layer
            bias=True,
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, segment_length, no_feature)
        )

        self.out = nn.Linear(n_hidden, n_class)

    def forward(self, x):
        r_out, (h_n, h_c) = self.lstm_layer(x.float(), None)
        r_out = F.dropout(r_out, 0.3)

        test_output = self.out(r_out[:, -1, :]) # choose r_out at the last time step
        return test_output

lstm = LSTM()
print(lstm)

optimizer = torch.optim.Adam(lstm.parameters(), lr=LR, weight_decay=l2)   # optimize all parameters
loss_func = nn.CrossEntropyLoss()

LSTM(
  (lstm_layer): LSTM(64, 128, num_layers=2, batch_first=True)
  (out): Linear(in_features=128, out_features=10, bias=True)
)


In [21]:
best_acc = []
best_auc = []

# training and testing
start_time = time.perf_counter()
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):

        output = lstm(train_x)  # LSTM output of training data
        loss = loss_func(output, train_y.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

    if epoch % 10 == 0:
        test_output = lstm(test_fea_norm1)  # LSTM output of test data
        test_loss = loss_func(test_output, test_label.long())

        test_y_score = one_hot(test_label.data.cpu().numpy())  # .cpu() can be removed if your device is cpu.
        pred_score = F.softmax(test_output, dim=1).data.cpu().numpy()  # normalize the output
        auc_score = roc_auc_score(test_y_score, pred_score)

        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()
        pred_train = torch.max(output, 1)[1].data.cpu().numpy()

        test_acc = accuracy_score(test_label.data.cpu().numpy(), pred_y)
        train_acc = accuracy_score(train_y.data.cpu().numpy(), pred_train)


        print('Epoch: ', epoch, '|train loss: %.4f' % loss.item(),
              ' train ACC: %.4f' % train_acc, '| test loss: %.4f' % test_loss.item(),
              'test ACC: %.4f' % test_acc, '| AUC: %.4f' % auc_score)
        best_acc.append(test_acc)
        best_auc.append(auc_score)

IndexError: Target 10 is out of bounds.

In [ ]:
current_time = time.perf_counter()
running_time = current_time - start_time
print(classification_report(test_label.data.cpu().numpy(), pred_y))
print('BEST TEST ACC: {}, AUC: {}'.format(max(best_acc), max(best_auc)))
print("Total Running Time: {} seconds".format(round(running_time, 2)))

## CNN

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(2,4),
                stride=1,
                padding= (1,2)  #([1,2]-1)/2,
            ),
            nn.ReLU(),
            nn.MaxPool2d((2,4))
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, (2,2), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        self.fc = nn.Linear(4*8*32, 128)  # 64*2*4
        self.out = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc(x))
        x = F.dropout(x, 0.2)

        output = self.out(x)
        return output, x

cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(2, 4), stride=(1, 1), padding=(1, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 4), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1024, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=2, bias=True)
)


In [23]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR, weight_decay=l2)
loss_func = nn.CrossEntropyLoss()

best_acc = []
best_auc = []

# training and testing
start_time = time.perf_counter()
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):

        output = cnn(train_x)[0]  # CNN output of training data
        loss = loss_func(output, train_y.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

    if epoch % 10 == 0:
        test_output = cnn(test_fea_norm1)[0]  # CNN output of test data
        test_loss = loss_func(test_output, test_label.long())

        test_y_score = one_hot(test_label.data.cpu().numpy())  # .cpu() can be removed if your device is cpu.
        pred_score = F.softmax(test_output, dim=1).data.cpu().numpy()  # normalize the output
        auc_score = roc_auc_score(test_y_score, pred_score)

        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()
        pred_train = torch.max(output, 1)[1].data.cpu().numpy()

        test_acc = accuracy_score(test_label.data.cpu().numpy(), pred_y)
        train_acc = accuracy_score(train_y.data.cpu().numpy(), pred_train)


        print('Epoch: ', epoch,  '|train loss: %.4f' % loss.item(),
              ' train ACC: %.4f' % train_acc, '| test loss: %.4f' % test_loss.item(),
              'test ACC: %.4f' % test_acc, '| AUC: %.4f' % auc_score)
        best_acc.append(test_acc)
        best_auc.append(auc_score)

RuntimeError: Given groups=1, weight of size [16, 1, 2, 4], expected input[1, 129759, 3, 64] to have 1 channels, but got 129759 channels instead

In [ ]:
current_time = time.perf_counter()
running_time = current_time - start_time
print(classification_report(test_label.data.cpu().numpy(), pred_y))
print('BEST TEST ACC: {}, AUC: {}'.format(max(best_acc), max(best_auc)))
print("Total Running Time: {} seconds".format(round(running_time, 2)))